In [2]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources  import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import UnstructuredURLLoader
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [3]:
llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY, temperature=0.5, max_output_tokens=250)

loaders= UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/sensex-up-100-pts-nifty-tests-24-550-it-pharma-fmcg-stocks-fall-small-midcaps-snap-7-day-streak-13210554.html",
    "https://www.moneycontrol.com/news/politics/will-stick-to-two-language-policy-reaffirms-karnataka-cm-siddaramaiah-after-maharashtra-row-13210905.html",
    "https://www.moneycontrol.com/news/india/eam-s-jaishankar-on-india-us-trade-deal-very-intricate-negotiation-there-will-be-some-give-and-take-13210171.html"
])

data= loaders.load()
len(data)

3

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

docs = text_splitter.split_documents(data)
len(docs)

29

In [5]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(docs, embeddings)

In [ ]:
vectorindex = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

In [ ]:
llm = GoogleGenerativeAI(model="gemini-2.0-flash", api_key=GOOGLE_API_KEY, temperature=0.5, max_output_tokens=250)

In [11]:
chain = RetrievalQAWithSourcesChain.from_llm(
    llm=llm, 
    retriever=vectorindex.as_retriever()
)

chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=GoogleGenerativeAI(model='gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.5, max_output_tokens=250, client=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.5, max_output_tokens=250, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000289251AEE10>, default_metadata=(), model_kwargs={})), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbos

In [13]:
import warnings
warnings.filterwarnings("ignore")

query= "What is the latest news on the Sensex and Nifty?"

langchain.debug = True

chain({"question": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is the latest news on the Sensex and Nifty?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Read more: Kalpataru shares list flat at IPO price on NSE; experts recommend buy, sell or hold strategy\n\nShares of Sigachi Industries slumped 7 percent, falling for a second straight session after the company temporarily shut its Telangana plant, where a blast on June 30 killed at least 34 people and injured several others. The stock was trading at Rs 45.42 in the morning, down nearly 18 percent over two days following the incident. The explosion reportedly occurred in a chemical reactor at the pharma unit in Pashamylaram.\n\n\"We see 25,450 on t

{'answer': 'The Sensex is up 100 points and the Nifty is testing 24,550. Key support levels are 25,450 on the Nifty and 83,500 on the Sensex. If these levels hold, a move towards 25,600/83,900 is likely, with the uptrend potentially extending to 25,670/84,100.\n',
 'sources': 'https://www.moneycontrol.com/news/business/markets/sensex-up-100-pts-nifty-tests-24-550-it-pharma-fmcg-stocks-fall-small-midcaps-snap-7-day-streak-13210554.html'}